In [2]:
from datetime import datetime, timedelta
from gmpacket.provenance import SoftwareAgent, PersonAgent, OrganizationAgent, Website, Provenance
from gmpacket.feature import (
    Feature,
    FeatureGeometry,
    FeatureProperties,
    Metric,
    MetricDimensions,
    MetricProperties,
    Stream,
    StreamHousing,
    StreamProperties,
    Trace,
    TraceProperties,
)
from gmpacket.packet import Event, GroundMotionPacket

# Dealing with Provenance

In [3]:
person = PersonAgent.from_params("Alex Processor", 
                                 "aprocessor@datagenerator.org", 
                                 "Data Processor")
person

PersonAgent(label='Alex Processor', name='Alex Processor', email='aprocessor@datagenerator.org', role='Data Processor', prov:type={'$': 'prov:Person', 'type': 'prov:QUALIFIED_NAME'})

In [4]:
website = Website(url="https://code.usgs.gov/ghsc/esi/groundmotion-processing/#introduction")
software = SoftwareAgent.from_params("gmprocess", "1.2.0", website)
software

SoftwareAgent(label='gmprocess', name='gmprocess', version='1.2.0', website=Website(url='https://code.usgs.gov/ghsc/esi/groundmotion-processing/#introduction', site_type='xsd:anyURI'), prov:type={'$': 'prov:SoftwareAgent', 'type': 'prov:QUALIFIED_NAME'})

In [5]:
org_website = Website(url="https://www.datagenerator.org")
organization = OrganizationAgent.from_params("Data Generator", "Data Provider", org_website)
organization

OrganizationAgent(label='Data Generator', name='Data Generator', role='Data Provider', website=Website(url='https://www.datagenerator.org', site_type='xsd:anyURI'), prov:type={'$': 'prov:Organization', 'type': 'prov:QUALIFIED_NAME'})

In [6]:
# the names of the keys are not important, but the values must be valid Person,Software, or 
# OrganizationAgent objects.
agents = {"person": person, "software": software, "organization":organization}
provenance = Provenance(agent=agents)
provenance

Provenance(agent={'person': PersonAgent(label='Alex Processor', name='Alex Processor', email='aprocessor@datagenerator.org', role='Data Processor', prov:type={'$': 'prov:Person', 'type': 'prov:QUALIFIED_NAME'}), 'software': SoftwareAgent(label='gmprocess', name='gmprocess', version='1.2.0', website=Website(url='https://code.usgs.gov/ghsc/esi/groundmotion-processing/#introduction', site_type='xsd:anyURI'), prov:type={'$': 'prov:SoftwareAgent', 'type': 'prov:QUALIFIED_NAME'}), 'organization': OrganizationAgent(label='Data Generator', name='Data Generator', role='Data Provider', website=Website(url='https://www.datagenerator.org', site_type='xsd:anyURI'), prov:type={'$': 'prov:Organization', 'type': 'prov:QUALIFIED_NAME'})}, prefix={'seis_prov': 'http://seisprov.org/seis_prov/0.1/#'})

## Creating Simple Metrics

In [7]:
pga_props = MetricProperties(description="Peak Ground Accleration",
                           name="PGA",
                           units= "g",
                           provenance_ids= provenance.getAgents())
pga_props

MetricProperties(description='Peak Ground Accleration', name='PGA', units='g', provenance_ids=['person', 'software', 'organization'], time_of_peak=None)

In [8]:
# for scalar values like PGA, MetricDimensions are not necessary
pga_metric = Metric(properties=pga_props, values=1.5)
pga_metric

Metric(properties=MetricProperties(description='Peak Ground Accleration', name='PGA', units='g', provenance_ids=['person', 'software', 'organization'], time_of_peak=None), dimensions=None, values=1.5)

# Creating Complex Metrics 

In [9]:
sa_props = MetricProperties(description="Spectral Accleration",
                             name="SA",
                             units= "g",
                             provenance_ids= provenance.getAgents())
sa_props

MetricProperties(description='Spectral Accleration', name='SA', units='g', provenance_ids=['person', 'software', 'organization'], time_of_peak=None)

In [10]:
sa_dims = MetricDimensions(number=2,
                           names=["critical damping", "period"],
                           units=["%","s"],
                           axis_values=[[5.0],[0.3, 1.0, 3.0]])
sa_dims

MetricDimensions(number=2, names=['critical damping', 'period'], units=['%', 's'], axis_values=[[5.0], [0.3, 1.0, 3.0]])

In [11]:
sa_metric = Metric(properties=sa_props, dimensions=sa_dims, values=[[1.2, 1.4, 1.6]])
sa_metric

Metric(properties=MetricProperties(description='Spectral Accleration', name='SA', units='g', provenance_ids=['person', 'software', 'organization'], time_of_peak=None), dimensions=MetricDimensions(number=2, names=['critical damping', 'period'], units=['%', 's'], axis_values=[[5.0], [0.3, 1.0, 3.0]]), values=[[1.2, 1.4, 1.6]])

# Creating Traces

In [12]:
end_time = datetime.utcnow()
start_time = end_time - timedelta(seconds=30)
sa_trace_props = TraceProperties(channel_code="HNE", 
                                 location_code="10", 
                                 as_recorded=True, 
                                 azimuth=90, 
                                 dip=0,
                                 start_time=start_time, 
                                 end_time=end_time)
sa_trace_props

TraceProperties(channel_code='HNE', location_code='10', as_recorded=True, azimuth=90.0, dip=0.0, start_time=datetime.datetime(2023, 2, 1, 16, 56, 7, 813536), end_time=datetime.datetime(2023, 2, 1, 16, 56, 37, 813536))

In [13]:
metrics = [pga_metric, sa_metric]
hne_trace = Trace(properties=sa_trace_props, metrics=metrics)
hne_trace

Trace(properties=TraceProperties(channel_code='HNE', location_code='10', as_recorded=True, azimuth=90.0, dip=0.0, start_time=datetime.datetime(2023, 2, 1, 16, 56, 7, 813536), end_time=datetime.datetime(2023, 2, 1, 16, 56, 37, 813536)), metrics=[Metric(properties=MetricProperties(description='Peak Ground Accleration', name='PGA', units='g', provenance_ids=['person', 'software', 'organization'], time_of_peak=None), dimensions=None, values=1.5), Metric(properties=MetricProperties(description='Spectral Accleration', name='SA', units='g', provenance_ids=['person', 'software', 'organization'], time_of_peak=None), dimensions=MetricDimensions(number=2, names=['critical damping', 'period'], units=['%', 's'], axis_values=[[5.0], [0.3, 1.0, 3.0]]), values=[[1.2, 1.4, 1.6]])])

# Creating a Stream

In [14]:
stream_housing = StreamHousing(cosmos_code=10, description="A building", stream_depth=0.0)
stream_housing

StreamHousing(cosmos_code=10, description='A building', stream_depth=0.0, stream_location=None)

In [15]:
stream_props = StreamProperties(band_code="H", 
                                instrument_code="N", 
                                samples_per_second=100.0, 
                                stream_housing=stream_housing)
stream_props

StreamProperties(band_code='H', instrument_code='N', samples_per_second=100.0, stream_housing=StreamHousing(cosmos_code=10, description='A building', stream_depth=0.0, stream_location=None))

In [16]:
stream = Stream(properties=stream_props, traces=[hne_trace])
stream

Stream(properties=StreamProperties(band_code='H', instrument_code='N', samples_per_second=100.0, stream_housing=StreamHousing(cosmos_code=10, description='A building', stream_depth=0.0, stream_location=None)), traces=[Trace(properties=TraceProperties(channel_code='HNE', location_code='10', as_recorded=True, azimuth=90.0, dip=0.0, start_time=datetime.datetime(2023, 2, 1, 16, 56, 7, 813536), end_time=datetime.datetime(2023, 2, 1, 16, 56, 37, 813536)), metrics=[Metric(properties=MetricProperties(description='Peak Ground Accleration', name='PGA', units='g', provenance_ids=['person', 'software', 'organization'], time_of_peak=None), dimensions=None, values=1.5), Metric(properties=MetricProperties(description='Spectral Accleration', name='SA', units='g', provenance_ids=['person', 'software', 'organization'], time_of_peak=None), dimensions=MetricDimensions(number=2, names=['critical damping', 'period'], units=['%', 's'], axis_values=[[5.0], [0.3, 1.0, 3.0]]), values=[[1.2, 1.4, 1.6]])])])

# Creating a Feature (station)

In [17]:
feature_geom = FeatureGeometry(coordinates=[-124.1, 32.0, 1.1])
feature_geom

FeatureGeometry(coordinates=[-124.1, 32.0, 1.1], type='Point')

In [18]:
feature_props = FeatureProperties(network_code="NC", 
                                  station_code="ABCD", 
                                  name="A nice place for picnics", 
                                  streams=[stream])
feature_props

FeatureProperties(network_code='NC', station_code='ABCD', name='A nice place for picnics', streams=[Stream(properties=StreamProperties(band_code='H', instrument_code='N', samples_per_second=100.0, stream_housing=StreamHousing(cosmos_code=10, description='A building', stream_depth=0.0, stream_location=None)), traces=[Trace(properties=TraceProperties(channel_code='HNE', location_code='10', as_recorded=True, azimuth=90.0, dip=0.0, start_time=datetime.datetime(2023, 2, 1, 16, 56, 7, 813536), end_time=datetime.datetime(2023, 2, 1, 16, 56, 37, 813536)), metrics=[Metric(properties=MetricProperties(description='Peak Ground Accleration', name='PGA', units='g', provenance_ids=['person', 'software', 'organization'], time_of_peak=None), dimensions=None, values=1.5), Metric(properties=MetricProperties(description='Spectral Accleration', name='SA', units='g', provenance_ids=['person', 'software', 'organization'], time_of_peak=None), dimensions=MetricDimensions(number=2, names=['critical damping', 'p

In [19]:
feature = Feature(geometry=feature_geom, properties=feature_props)
feature

Feature(geometry=FeatureGeometry(coordinates=[-124.1, 32.0, 1.1], type='Point'), properties=FeatureProperties(network_code='NC', station_code='ABCD', name='A nice place for picnics', streams=[Stream(properties=StreamProperties(band_code='H', instrument_code='N', samples_per_second=100.0, stream_housing=StreamHousing(cosmos_code=10, description='A building', stream_depth=0.0, stream_location=None)), traces=[Trace(properties=TraceProperties(channel_code='HNE', location_code='10', as_recorded=True, azimuth=90.0, dip=0.0, start_time=datetime.datetime(2023, 2, 1, 16, 56, 7, 813536), end_time=datetime.datetime(2023, 2, 1, 16, 56, 37, 813536)), metrics=[Metric(properties=MetricProperties(description='Peak Ground Accleration', name='PGA', units='g', provenance_ids=['person', 'software', 'organization'], time_of_peak=None), dimensions=None, values=1.5), Metric(properties=MetricProperties(description='Spectral Accleration', name='SA', units='g', provenance_ids=['person', 'software', 'organizatio

# Creating a Ground Motion Packet

In [20]:
event = Event.from_params("us2023abcd", datetime.utcnow(), 7.3, 32.1, -120.0, 35.3)
event

Event(type='Feature', properties={'id': 'us2023abcd', 'time': datetime.datetime(2023, 2, 1, 16, 56, 37, 863269), 'magnitude': 7.3}, geometry={'type': 'Point', 'coordinates': [-120.0, 32.1, 35300.0]})

In [21]:
gm_packet = GroundMotionPacket(version="0.1dev", 
                               event=event, 
                               provenance=provenance, 
                               features=[feature]
                              )
gm_packet

GroundMotionPacket(version='0.1dev', creation_time=datetime.datetime(2023, 2, 1, 16, 56, 37, 732084), event=Event(type='Feature', properties={'id': 'us2023abcd', 'time': datetime.datetime(2023, 2, 1, 16, 56, 37, 863269), 'magnitude': 7.3}, geometry={'type': 'Point', 'coordinates': [-120.0, 32.1, 35300.0]}), provenance=Provenance(agent={'person': PersonAgent(label='Alex Processor', name='Alex Processor', email='aprocessor@datagenerator.org', role='Data Processor', prov:type={'$': 'prov:Person', 'type': 'prov:QUALIFIED_NAME'}), 'software': SoftwareAgent(label='gmprocess', name='gmprocess', version='1.2.0', website=Website(url='https://code.usgs.gov/ghsc/esi/groundmotion-processing/#introduction', site_type='xsd:anyURI'), prov:type={'$': 'prov:SoftwareAgent', 'type': 'prov:QUALIFIED_NAME'}), 'organization': OrganizationAgent(label='Data Generator', name='Data Generator', role='Data Provider', website=Website(url='https://www.datagenerator.org', site_type='xsd:anyURI'), prov:type={'$': 'pr

# Exporting GroundMotion Packets

In [22]:
gm_packet.as_json()

'{"version":"0.1dev","creation_time":"2023-02-01T16:56:37Z","event":{"type":"Feature","properties":{"id":"us2023abcd","time":"2023-02-01T16:56:37Z","magnitude":7.3},"geometry":{"type":"Point","coordinates":[-120,32.1,35300]}},"provenance":{"agent":{"person":{"prov:label":"Alex Processor","seis_prov:name":"Alex Processor","seis_prov:email":"aprocessor@datagenerator.org","seis_prov:role":"Data Processor","prov:type":{"$":"prov:Person","type":"prov:QUALIFIED_NAME"}},"software":{"prov:label":"gmprocess","seis_prov:software_name":"gmprocess","seis_prov:software_version":"1.2.0","seis_prov:website":{"$":"https://code.usgs.gov/ghsc/esi/groundmotion-processing/#introduction","type":"xsd:anyURI"},"prov:type":{"$":"prov:SoftwareAgent","type":"prov:QUALIFIED_NAME"}},"organization":{"prov:label":"Data Generator","seis_prov:name":"Data Generator","seis_prov:role":"Data Provider","seis_prov:website":{"$":"https://www.datagenerator.org","type":"xsd:anyURI"},"prov:type":{"$":"prov:Organization","type"

In [23]:
df = gm_packet.to_dataframe()
print(df)

           id                       time  magnitude  event_longitude  \
0  us2023abcd 2023-02-01 16:56:37.863269        7.3           -120.0   
1  us2023abcd 2023-02-01 16:56:37.863269        7.3           -120.0   

   event_latitude  event_depth network station              station_name  \
0            32.1      35300.0      NC    ABCD  A nice place for picnics   
1            32.1      35300.0      NC    ABCD  A nice place for picnics   

   station_longitude  station_latitude  station_elevation channel location  \
0             -124.1              32.0                1.1     HNE       10   
1             -124.1              32.0                1.1     HNE       10   

   PGA(g)  SA(g)_critical_damping_5.0%_period_0.3s  \
0     1.5                                      1.2   
1     1.5                                      1.2   

   SA(g)_critical_damping_5.0%_period_1.0s  \
0                                      1.4   
1                                      1.4   

   SA(g)_critical